# Code 5: Building Damage Detection with Convolutional Neural Networks (CNNs)

This code demonstrates the process of building damage detection using Convolutional Neural Networks (CNNs). It involves preprocessing images and annotations, creating and training a CNN model (based on VGG16), performing data augmentation, and utilizing selective search to identify and visualize regions of interest (potential damaged areas) in test images. The code showcases model training, evaluation, and testing for identifying whether an image contains signs of building damage.

## Part 1: Imports and Working directory

In [ ]:
import os, cv2, keras
from cv2.ximgproc import segmentation
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Dense
from keras import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.models import load_model

path = "path to train images"
annot = "path to corresponding annotations files"


## Part 2: Display Annotated Images

In [ ]:
# Process the first ten files in the annotation directory.
for e, i in enumerate(os.listdir(annot)):
    if e < 10: # Limit the loop to the first ten files
        filename = i.split(".")[0] + ".jpg"
        print(filename)

        # Load the image and corresponding annotation data.
        img = cv2.imread(os.path.join(path, filename))
        df = pd.read_csv(os.path.join(annot, i))

        # Display the original image.
        # plt.imshow(img)

        # Iterate through annotation data and draw bounding boxes.
        for row in df.iterrows():
            x1 = int(row[1][4])
            y1 = int(row[1][5])
            x2 = int(row[1][6])
            y2 = int(row[1][7])
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)

        # Display the annotated image in a separate figure.
        # plt.figure()
        # plt.imshow(img)

        # Exit the loop after processing the first image.
        break

## Part 3: Selective Seach and Bounding Box

In [ ]:
# Enable OpenCV optimizations for improved performance.
cv2.setUseOptimized(True);

# Create an instance of the Selective Search Segmentation algorithm.
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

# Load an image from the specified path.
im = cv2.imread(os.path.join(path, "clip_1.jpg"))

# Set the base image for the Selective Search algorithm.
ss.setBaseImage(im)

# Switch to the fast mode of Selective Search.
ss.switchToSelectiveSearchFast()

# Perform Selective Search to generate region proposals.
rects = ss.process()

# Create a copy of the image for visualization.
imOut = im.copy()

# Iterate through the generated rectangles and draw bounding boxes.
for i, rect in (enumerate(rects)):
    x, y, w, h = rect
    #     print(x,y,w,h)
    #     imOut = imOut[x:x+w,y:y+h]
    cv2.rectangle(imOut, (x, y), (x + w, y + h), (0, 255, 0), 1, cv2.LINE_AA)

# Display the image with drawn bounding boxes.
# plt.figure()
# plt.imshow(imOut)

## Part 4: Data Preparation and Augmentation

In [ ]:
# Initialize empty lists for storing training images and labels.
train_images = []
train_labels = []

# Define a function to calculate Intersection over Union (IoU) between two bounding boxes.
def get_iou(bb1, bb2):
    """
        Calculates the Intersection over Union (IoU) between two bounding boxes.

        Args:
            bb1 (dict): Dictionary containing bounding box coordinates with keys 'x1', 'y1', 'x2', and 'y2'.
            bb2 (dict): Dictionary containing bounding box coordinates with keys 'x1', 'y1', 'x2', and 'y2'.

        Returns:
            float: IoU value between 0 and 1, indicating the overlap between the two bounding boxes.
        """
    # Assertions to validate input bounding boxes
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # Calculate intersection area
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    # Check for non-overlapping bounding boxes
    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # Calculate IoU
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)

    # Ensure IoU value is within the valid range [0, 1]
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

def non_max_suppression(boxes, probs, overlap_thresh):
    """
       Performs non-maximum suppression to filter out redundant bounding boxes.

       Args:
           boxes (list): List of dictionaries representing bounding box coordinates.
           probs (list): List of confidence scores corresponding to the bounding boxes.
           overlap_thresh (float): Threshold for determining overlapping bounding boxes.

       Returns:
           list: Indexes of the selected bounding boxes after non-maximum suppression.
       """
    # Sort the bounding boxes by their confidence scores in descending order
    idxs = np.argsort(probs)[::-1]

    # Initialize the list of picked indexes
    pick = []

    while len(idxs) > 0:
        # Get the index of the highest confidence bounding box
        i = idxs[0]

        # Add the index to the list of picked indexes
        pick.append(i)

        # Calculate the overlap of this bounding box with others
        overlap = [get_iou(boxes[i], boxes[j]) for j in idxs[1:]]  # Calculate IOUs with other bounding boxes

        # Remove indexes of overlapping bounding boxes
        idxs = idxs[np.where(np.array(overlap) <= overlap_thresh)[0] + 1]

    return pick

# Loop through the files in the annotation directory with enumeration.
for e, i in enumerate(os.listdir(annot)):
    try:
        # Check if the file name starts with "clip".
        if i.startswith("clip"):
            filename = i.split(".")[0] + ".jpg"
            print(e, filename)

            # Read the image and annotation data.
            image = cv2.imread(os.path.join(path, filename))
            df = pd.read_csv(os.path.join(annot, i))

            # Initialize a list to store ground truth bounding box values.
            gtvalues = []

            # Extract ground truth bounding box coordinates from the DataFrame.
            for row in df.iterrows():
                for row in df.iterrows():
                    x1 = int(row[1][4])
                    y1 = int(row[1][5])
                    x2 = int(row[1][6])
                    y2 = int(row[1][7])
                gtvalues.append({"x1": x1, "x2": x2, "y1": y1, "y2": y2})

            # Set the base image for Selective Search and perform fast mode segmentation.
            ss.setBaseImage(image)
            ss.switchToSelectiveSearchFast()
            ssresults = ss.process()

            # Create a copy of the image for visualization.
            imout = image.copy()

            # Initialize counters and flags. (Helps to manage the dataset generation process efficiently
            # while keeping a balance between positive and negative samples for training an object detection model)
            counter = 0
            falsecounter = 0
            flag = 0
            fflag = 0
            bflag = 0

            # Loop through the selective search results and process bounding boxes.
            for e, result in enumerate(ssresults):
                if e < 2000 and flag == 0:
                    for gtval in gtvalues:
                        x, y, w, h = result
                        # Calculate IoU between result bounding box and ground truth.
                        iou = get_iou(gtval, {"x1": x, "x2": x + w, "y1": y, "y2": y + h})

                        # Process bounding boxes for training dataset.
                        if counter < 30:
                            if iou > 0.70:
                                timage = imout[y:y + h, x:x + w]
                                resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
                                train_images.append(resized)
                                train_labels.append(1)
                                counter += 1
                        else:
                            fflag = 1
                        if falsecounter < 30:
                            if iou < 0.3:
                                timage = imout[y:y + h, x:x + w]
                                resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
                                train_images.append(resized)
                                train_labels.append(0)
                                falsecounter += 1
                        else:
                            bflag = 1

                    # Set flags to exit the loops if necessary.
                    if fflag == 1 and bflag == 1:
                        print("inside")
                        flag = 1
    except Exception as e:
        print(e)
        print("error in " + filename)
        continue

# Convert the list of training images and labels into a NumPy array.
X_new = np.array(train_images)
y_new = np.array(train_labels)
# Display the shape of the newly created training data array.
X_new.shape


## Part 5: VGG16 Model and Training

In [ ]:

# Load VGG16 model with pre-trained weights
vggmodel = VGG16(weights='imagenet', include_top=True)
vggmodel.summary()

# Freeze layers up to layer 15
for layers in vggmodel.layers[:15]:
    layers.trainable = False

# Define custom output layer
X = vggmodel.layers[-2].output
predictions = Dense(2, activation="softmax")(X)

# Create the final model
model_final = Model(inputs=vggmodel.input, outputs=predictions)

# Compile the model
opt = Adam(learning_rate=0.0001)
model_final.compile(loss=keras.losses.categorical_crossentropy, optimizer=opt, metrics=["accuracy"])
model_final.summary()


# function to binarize training data
class MyLabelBinarizer(LabelBinarizer):
    def transform(self, y):
        Y = super().transform(y)
        if self.y_type_ == 'binary':
            return np.hstack((Y, 1 - Y))
        else:
            return Y

    def inverse_transform(self, Y, threshold=None):
        if self.y_type_ == 'binary':
            return super().inverse_transform(Y[:, 0], threshold)
        else:
            return super().inverse_transform(Y, threshold)


# apply Binarizer
lenc = MyLabelBinarizer()
Y = lenc.fit_transform(y_new)

# split training data for testing and look at their dimensions
X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.20)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


# Data augmentation
aug = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    zoom_range=0.2,
    shear_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode="nearest"
)

BS = 40
EPOCHS = 5

# Train the model
print("[INFO] training head...")
hist = model_final.fit(
    aug.flow(X_train, y_train, batch_size=BS),
    steps_per_epoch=len(X_train) // BS,
    validation_data=(X_test, y_test),
    validation_steps=len(X_test) // BS,
    epochs=EPOCHS
)


model.save("path to model.h5")
model = keras.models.load_model("")



## Part 6: Plotting and Testing

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), hist.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), hist.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), hist.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), hist.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()
plt.savefig("plot")

# test on test images
for im in X_test:
    img = np.expand_dims(im, axis=0)
    out = model.predict(img)
    if out[0][0] > out[0][1]:
        print("destroyed")
    else:
        print("not destroyed")


## Part 7: Loading Model and Processing Images

In [ ]:

model_final = load_model("")
# Initialize a counter to keep track of processed images.
z = 0

# 3 clips kopiert und umbenannt zu "test_"
# Loop through the files in the directory 'path'.
for e, i in enumerate(os.listdir(path)):
    if i.startswith("test_"):
        z += 1
        img = cv2.imread(os.path.join(path, i))
        ss.setBaseImage(img)
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process() #Perform Selective Search on the image.
        imout = img.copy() #Create a copy of the image for drawing bounding boxes.
        boxes = []

        # Iterate through the Selective Search results.
        for e, result in enumerate(ssresults):
            if e < 2000:
                x, y, w, h = result
                boxes.append([x, y, x + w, y + h])

                # Extract the region of interest and resize it.
                timage = imout[y:y + h, x:x + w]
                resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
                img = np.expand_dims(resized, axis=0)

        # Convert boxes to a NumPy array
        boxes = np.array(boxes)

        # After predicting bounding boxes and their probabilities
        out = model_final.predict(img)
        proba = out[:, 0]

        imout_copy = imout.copy()  # Create a copy for drawing boxes

        # Apply Non-Maximum Suppression
        overlap_thresh = 0.3  # Adjust this threshold as needed
        nms_idxs = non_max_suppression(boxes, proba, overlap_thresh)
        filtered_boxes = boxes[nms_idxs]

        # Draw rectangles for filtered bounding boxes
        for i in nms_idxs:
            x, y, w, h = filtered_boxes[i]
            cv2.rectangle(imout_copy, (x, y), (x + w, y + h), (0, 255, 0), 1, cv2.LINE_AA)

        import matplotlib.pyplot as plt

        # Save the image with rectangles
        plt.figure()
        plt.imshow(imout_copy)

        # Generate a unique filename based on the iteration index (z)
        downloaded_filename = f"path_{z}.png"

        # Save the image to the generated filename
        plt.savefig(downloaded_filename)

        # Close the plot to prevent unwanted display
        plt.close()

    print("Image processing and saving completed.")